In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import cosima_cookbook as cc
import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


netcdf_index loaded.
Available exptdata keys:  ['1deg', '025deg', '01deg']


AttributeError: module 'cosima_cookbook' has no attribute 'database'

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:34840 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
# input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path = '/g/data/e14/erd561/Australia_3/'

In [8]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
time_units = exptdata.exptdict[ekey]['time_units']
n_files = exptdata.exptdict[ekey]['n_files']
offset = exptdata.exptdict[ekey]['offset']
print(expt)
print(time_units)
print(n_files)
print(offset)

sea_level_z0_01_mod_month = cc.get_nc_variable(
    expt,'ocean_daily.nc','eta_t',time_units=time_units,n=n_files,offset=offset).\
                            sel(yt_ocean=arange(latS,latN,0.5), method='nearest').\
                            sel(xt_ocean=arange(lonW,lonE,0.5), method='nearest')
print(sea_level_z0_01_mod_month)

sea_level_z0_01_mod_month.load()
print(sea_level_z0_01_mod_month)

xt_ocean = sea_level_z0_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
sea_level_z0_01_mod_month = sea_level_z0_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(sea_level_z0_01_mod_month)

sea_level_z0_01_mod = sea_level_z0_01_mod_month.groupby('time.year').mean('time')
print(sea_level_z0_01_mod)

sea_level_z0_01_mod = sea_level_z0_01_mod.sel(year=slice(1993,2017))
print(sea_level_z0_01_mod)

<xarray.DataArray 'eta_t' (year: 34, yt_ocean: 181, xt_ocean: 181)>
array([[[      nan,       nan, ..., -1.554744, -1.555086],
        [      nan,       nan, ..., -1.552836, -1.552635],
        ...,
        [ 0.428359,  0.423537, ...,  0.608999,  0.591775],
        [ 0.443136,  0.470953, ...,  0.612494,  0.596341]],

       [[      nan,       nan, ..., -1.584716, -1.584451],
        [      nan,       nan, ..., -1.589159, -1.589877],
        ...,
        [ 0.389747,  0.411499, ...,  0.645868,  0.641956],
        [ 0.418304,  0.430131, ...,  0.632884,  0.634457]],

       ...,

       [[      nan,       nan, ..., -1.57397 , -1.572852],
        [      nan,       nan, ..., -1.586361, -1.586138],
        ...,
        [ 0.505984,  0.506387, ...,  0.632004,  0.634077],
        [ 0.515525,  0.517584, ...,  0.619936,  0.616379]],

       [[      nan,       nan, ..., -1.59033 , -1.591264],
        [      nan,       nan, ..., -1.606643, -1.610039],
        ...,
        [ 0.463419,  0.469171, ...,

In [10]:
year = sea_level_z0_01_mod.year

lat_01_mod = sea_level_z0_01_mod.yt_ocean
lon_01_mod = sea_level_z0_01_mod.xt_ocean

sea_level_z0_01_trans_mod = sea_level_z0_01_mod
print(np.shape(sea_level_z0_01_trans_mod))
sea_level_z0_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='sea_level_z0_01')
sea_level_z0_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='sea_level_z0_01')
sea_level_z0_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='sea_level_z0_01')
for iid, i in enumerate(lat_01_mod):
    for jid, j in enumerate(lon_01_mod):
        sea_level_z0_01_slope_mod[iid, jid], _, _, sea_level_z0_01_p_value_mod[iid, jid], sea_level_z0_01_std_err_mod[iid, jid] = \
        stats.linregress(year, sea_level_z0_01_trans_mod[:, iid, jid])
    print('lat_01_mod ' + str(np.array(i)))
print(sea_level_z0_01_slope_mod)

(25, 181, 181)
lat_01_mod -69.99968147628742
lat_01_mod -69.49280922372161
lat_01_mod -68.9859369711558
lat_01_mod -68.47906471858998
lat_01_mod -68.01443182040465
lat_01_mod -67.50755956783884
lat_01_mod -67.00068731527303
lat_01_mod -66.49381506270721
lat_01_mod -65.9869428101414
lat_01_mod -65.48007055757559
lat_01_mod -65.01543765939026
lat_01_mod -64.50375411947216
lat_01_mod -63.98230575427869
lat_01_mod -63.49561163694184
lat_01_mod -63.00048378360056
lat_01_mod -62.496813324719064
lat_01_mod -61.98449280805486
lat_01_mod -61.51115045088618
lat_01_mod -60.98201976395361
lat_01_mod -60.49322107491858
lat_01_mod -59.996941213465746
lat_01_mod -59.49310348481155
lat_01_mod -58.981632346702604
lat_01_mod -58.51472041667825
lat_01_mod -57.98854233110828
lat_01_mod -57.50827664011144
lat_01_mod -57.02160573730968
lat_01_mod -56.47328717333905
lat_01_mod -55.972931534839404
lat_01_mod -55.52266844284527
lat_01_mod -55.00989047675615
lat_01_mod -54.49047104792663
lat_01_mod -54.02315489

In [11]:
sea_level_z0_01_mod_xr = xr.DataArray(sea_level_z0_01_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='sea_level_z0_01_mod',
                        coords=[lat_01_mod, lon_01_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(sea_level_z0_01_mod_xr)
sea_level_z0_01_mod_xr.to_netcdf(output_path + 'sea_level_z0_01_mod.nc')

sea_level_z0_01_slope_mod_xr = xr.DataArray(sea_level_z0_01_slope_mod, name='sea_level_z0_01_slope_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(sea_level_z0_01_slope_mod_xr)
sea_level_z0_01_slope_mod_xr.to_netcdf(output_path + 'sea_level_z0_01_slope_mod.nc')

sea_level_z0_01_p_value_mod_xr = xr.DataArray(sea_level_z0_01_p_value_mod, name='sea_level_z0_01_p_value_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(sea_level_z0_01_p_value_mod_xr)
sea_level_z0_01_p_value_mod_xr.to_netcdf(output_path + 'sea_level_z0_01_p_value_mod.nc')

sea_level_z0_01_std_err_mod_xr = xr.DataArray(sea_level_z0_01_std_err_mod, name='sea_level_z0_01_std_err_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(sea_level_z0_01_std_err_mod_xr)
sea_level_z0_01_std_err_mod_xr.to_netcdf(output_path + 'sea_level_z0_01_std_err_mod.nc')

<xarray.DataArray 'sea_level_z0_01_mod' (yt_ocean: 181, xt_ocean: 181, year: 25)>
array([[[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [-1.555117, -1.538673, ..., -1.57397 , -1.59033 ],
        [-1.553023, -1.53793 , ..., -1.572852, -1.591264]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [-1.566397, -1.552264, ..., -1.586361, -1.606643],
        [-1.566774, -1.554495, ..., -1.586138, -1.610039]],

       ...,

       [[ 0.429863,  0.386757, ...,  0.505984,  0.463419],
        [ 0.432882,  0.389582, ...,  0.506387,  0.469171],
        ...,
        [ 0.628294,  0.63791 , ...,  0.632004,  0.70038 ],
        [ 0.624128,  0.636428, ...,  0.634077,  0.696918]],

       [[ 0.443091,  0.411654, ...,  0.515525,  0.473256],
        [ 0.44223 ,  0.41309 , ...,  0.517584,  0.479172],
        ...,
        [ 0.626813,  